In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
from difflib import SequenceMatcher
from itertools import chain 
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import pickle

In [2]:
#: Get data and split into test and training data.
# Divide these parts into input and output
# Create output: Win, Draw, Defeat Create array.
# [1,0,0]= Win, [0,1,0]=Draw, [0,0,1]= Defeat

#Training divide 3 parts, 1. time sequences, 2. auxila'ry data, 3. output

#time sequences: statistics from both of the teams playing, i.e.
#max. last 5 games of [teamA,teamB] <- per past game
#this will be given into LSTM 

#auxilary data: Data about the current game, like table values, odds,player/team values.

#output: Win, Draw, Defeat for respective game

# part 1 and part 2 form x, part1= LSTM, part2= other Dense, then concatinate
# Part 3 forms y


#print(pd.read_pickle("Daten161718.pkl")[0][1][8])
#print(pd.read_pickle("Daten161718.pkl")[1])


In [3]:
"""
input: matchday number to be predicted
get the data from the match from each team
get training and test data
output: training, test data, teams
"""
def getdataTraining(nummer):
    data=pd.read_pickle("Daten161718.pkl")
    matrix= data[0]
    mannschaften= data[1]
    schleifennummer= nummer-1
    training=[]
    test=[]
    
    #Alle Spiele vom 1. Spieltag bis zum x. Spieltag holen
    #in Trainings und Testdaten zuordnen
    for mannschaft in range(0,20):
        for spieltag in range(0,schleifennummer):
            training.append(matrix[mannschaft][spieltag])
        test.append(matrix[mannschaft][nummer-1])
    
    #Spiele der Mannschaften zuordnen, ersten x Spiele gehören Team 1 usw.
    training_dic={}
    for i in range(0,20):
        training_dic[i]= training[(nummer-1)*i:(nummer-1)*(i+1)]
    
    return training_dic,test,mannschaften

"""
get teams from training set
output: teams
"""
def get_mannschaftenTraining():
    data=pd.read_pickle("Daten161718.pkl")
    mannschaften= data[1]
    return mannschaften

"""
input number
get data of seasons
output: traning test data, teams
"""
def getdata(nummer):
    data=pd.read_pickle("Daten2019.pkl")
    matrix= data[0]
    mannschaften= data[2]
    schleifennummer= nummer-1
    training=[]
    test=[]
    
    #Alle Spiele vom 1. Spieltag bis zum x. Spieltag holen
    #in Trainings und Testdaten zuordnen
    for mannschaft in range(0,18):
        for spieltag in range(0,schleifennummer):
            training.append(matrix[mannschaft][spieltag])
        test.append(matrix[mannschaft][nummer-1])
    
    #Spiele der Mannschaften zuordnen, ersten x Spiele gehören Team 1 usw.
    training_dic={}
    for i in range(0,18):
        training_dic[i]= training[(nummer-1)*i:(nummer-1)*(i+1)]
    
    return training_dic,test,mannschaften

"""
input: team1
get names of teams (opponent)
output: teams
"""
def get_mannschaften(name):
    data=pd.read_pickle(name)
    mannschaften= data[2]
    return mannschaften

"""
input match day
Auxiliary method to determine the number of matches and the time size.
Input: Desired match day, which should be predicted
Wanted to consider 5 games out of 5 games.
if this is not possible, less games will be considered for one game
worst-case: no 5 games can be considered and therefore no 5 games, but 2 and 1
output: time_size, number

"""
def size_of_time(spieltag):
    
    if spieltag >=12:
        time_size= 5
        anzahl= spieltag-6
    else:
        #spieltag=spieltag-1
        differenz1= spieltag-6
        differenz2= spieltag-5
        if differenz1 >=3: #Spieltag 9,10,11
            time_size= differenz1
            anzahl=5
        elif differenz2 ==3: #Spieltag 8
            time_size= differenz2
            anzahl=4
        elif spieltag-4==3:# Spieltag7
            time_size= 3
            anzahl= 3
        elif spieltag-3==3: # Spieltag6
            time_size= 3
            anzahl= 3
        elif spieltag-2==3: #Spieltag5
            time_size= 3
            anzahl= 3
        elif spieltag-1==3: # Spieltag 4
            time_size= 2
            anzahl= 2
        elif spieltag >2:
            time_size=1
            anzahl=1
        else:
            time_size=0
            anzahl=0
    return time_size,anzahl

"""
input; data, team
filter the statistics of the respective team
output: List of statistics from the games
"""
def get_statistik(data, team):
    liste=[]
    for i in data:
        teams= i[-2:]
        werte= i[:-2]
        if teams[0]==team:
            liste.append(werte[19:32])
        else:
            liste.append(werte[51:64])
    return liste

"""
input: team, list of teams
Similarity search of the teams
ouput: right team name
"""
def similar(mannschaft, liste): 
    wert=0
    currentteam=""
    for team in liste:
        similarity= SequenceMatcher(None, mannschaft, team).ratio()
        #print(team, similarity)
        if wert < similarity:
            wert= similarity
            currentteam= team
            
    return currentteam

"""
input: data, team, list of team
Find out the opponent of the passed team
Go through data and find team. There is also the opponentID
output: Name of opponent and location(is team home or visiting team).
"""
def getgegner(data,teamname,mannschaftsliste):
    current_team=similar(teamname,mannschaftsliste)#
    key= mannschaftsliste.index(current_team)
    for i in data:
        teams= i[-2:]
        if key in teams:
            current_teams= teams
            break
    #print(current_teams)
    if list(current_teams).index(key)==0:
        ort= "home"
    else:
        ort= "away"
    gegner= current_teams[current_teams!=key][0]
    gegnername= mannschaftsliste[int(gegner)]
    return gegnername,ort

"""
input: data, lis of team, team, matchday number, promoted team, relegated team
Reduction of the data
If there is a promoted team at the queried game
you can't look at the previous season, because there is only one team there
So --> remove data of the previous season
If there is a relegated team at the queried game
--> Error message
This is done for 3 seasons(so 2 promoted/relegated)
output: Data, new game day(instead of 40, 40-34=6).
"""
def data_auf_absteiger(data,mannschaftsliste,mannschaft,spieltag,absteiger,aufsteiger):

    current_team1=similar(mannschaft[0],mannschaftsliste)
    index1= mannschaftsliste.index(current_team1)
    
    current_team2=similar(mannschaft[1],mannschaftsliste)
    index2= mannschaftsliste.index(current_team2)
    index_liste=[index1,index2]
    found=0
    #print(index1,index2)
    #print(list(aufsteiger.values()),absteiger.values())
    auf= list(chain.from_iterable(list(aufsteiger.values())))
    ab= list(chain.from_iterable(list(absteiger.values())))
    if spieltag >34 and (index1 in auf or index2 in auf or index1 in ab or index2 in ab):
        
        if spieltag <69:
            del absteiger[1]
            del aufsteiger[2]
            #Schauen ob Mannschaft Auf-/Absteiger ist, zweite Saison
            for key,values in absteiger.items():
                if index1 in values or index2 in values:
                    return "KEY ERROR",""
            for key,values in aufsteiger.items():
                if index1 in values or index2 in values:
                    for key, values in data.items():
                        data.update({key:values[34:]})
                    return data, spieltag-34
                else: 
                    return data, spieltag
        #Schauen ob Mannschaft Auf/Absteiger ist, dritte Saison
        if spieltag >=69:
            del absteiger[0]
            for key,values in absteiger.items():
                if index1 in values or index2 in values:
                    return "KEY ERROR",""
            for key,values in aufsteiger.items():
                if index1 in values or index2 in values:
                    if key==1:
                        for key, values in data.items():
                            data.update({key:values[68:]})
                            
                        return data, spieltag-68
                    else:
                        for key, values in data.items():
                            data.update({key:values[68:]})
                        return data, spieltag-68
    else:
        return data, spieltag

"""
input: data, list of team, team, match day
Create time sequences
In the best case the last 5 game statistics are considered per game
How many are looked at is calculated with size_of_time
Now get x statistics from the team and its opponents
Save them in a list and consider the next game day
output: List of TimeSequences, size of each TimeSequence, team key.
"""
def time_sequence(data,mannschaftsliste,mannschaft,spieltag):
    
    time_size, anzahl= size_of_time(spieltag)
    time_series={}
    list_of_game_series=[]
    #print(time_size,anzahl)
    #Mannschaftnummer finden
    current_team=similar(mannschaft,mannschaftsliste)
    for key,values in data.items():
        start= spieltag-time_size-2
                
        if key!= mannschaftsliste.index(current_team):
            continue
        else:
            for i in range(time_size,spieltag-1): #Spiele die betrachtet werden
                teams= values[i][-2:]
                start_gegner= i-time_size
            
                if teams[0]==key:
                    #"Rohe" Daten holen
                    gegner_statistik= data[teams[1]][start_gegner:i]
                    team_statistik= data[teams[0]][start_gegner:i]
                    #Filterung der Statistiken
                    team_statistik= get_statistik(team_statistik,teams[0])
                    gegner_statistik= get_statistik(gegner_statistik,teams[1])
                    #Zusammenführung
                    game_series= np.concatenate((team_statistik, gegner_statistik), axis=1)
                else:
                    #"Rohe" Daten holen
                    gegner_statistik= data[teams[0]][start_gegner:i]
                    team_statistik= data[teams[1]][start_gegner:i]              
                    #Filterung der Statistiken
                    team_statistik= get_statistik(team_statistik,teams[1])
                    gegner_statistik= get_statistik(gegner_statistik,teams[0])
                    #Zusammenführung
                    game_series= np.concatenate((gegner_statistik, team_statistik), axis=1)
                #print(game_series)
                list_of_game_series.append(game_series)
    mannschaftskey= mannschaftsliste.index(current_team)
    return list_of_game_series, mannschaftskey

"""
input: data, key of team, matchday
Create auxiliary data, i.e. table, team, player values, betting odds.
extract output for these matches as well
se #size of time to find considered games
create list for aux_data and output
output: list of aux data and list of output values
"""
def auxilary_data(data,mannschaftskey,spieltag):
    time_size, anzahl= size_of_time(spieltag)
    aux_data={}
    out_data={}
    for key,values in data.items():
        #start= spieltag-time_size-2
        list_of_game_series=[]
        list_of_outputs=[]
        if key!= mannschaftskey:
            continue
        else:
            for i in range(time_size,spieltag-1):
                teams= values[i][-2:]
                start_gegner= i-time_size
                #print(values[i])
                home_team= values[i][:19]
                away_team= values[i][32:51]
                odds= values[i][64:67]
                output= values[i][67:70]
                #print(values[i][:19],values[i][32:51],values[i][64:67],values[i][67:70])
                game_data= np.concatenate((home_team,away_team,odds))
                list_of_outputs.append(output)
                list_of_game_series.append(game_data)
            aux_data[key]=list_of_game_series
            out_data[key]=list_of_outputs
            
    return list(aux_data.values()),list(out_data.values())

"""
input: trainings data, test data, team, match day
Create TimeSequence for the test data
Difference one looks only at the one game
First find out home and away team
Then create the TimeSequence for both teams
loop does not run, because the game is twice in the list
output: array with time sequences
"""
def time_sequence_test(data_training,data_test,team,spieltag):

    time_size, anzahl= size_of_time(spieltag)
    for spiele in data_test:
        mannschaften= spiele[-2:]
        if team in mannschaften:
            if mannschaften[0]==team:
                gegner= [mannschaften[1],"away"]
            else:
                gegner= [mannschaften[0],"home"]
            break
    liste=[]
    for key,values in data_training.items():
        
        if key!= team:
            continue
        else:

            gegner_statistik= data_training[gegner[0]][-time_size:]
            team_statistik= data_training[team][-time_size:]
            #Filterung der Statistiken
            team_statistik= get_statistik(team_statistik,team)
            gegner_statistik= get_statistik(gegner_statistik,gegner[0])
            #Zusammenführung
            if gegner[1]=="away":
                game_series= np.concatenate((team_statistik, gegner_statistik), axis=1)
            else:
                game_series= np.concatenate((gegner_statistik, team_statistik), axis=1)
            #print(values, time_size)
            liste.append(game_series)
            return liste

"""
input: test data, team
Fetch auxilary data for test
Simply fetch these values for a game
loop does not run because the game is in the list twice
output: List of Aux_Data and list of playing teams.
"""
def auxilary_data_test(data_test,team):
    liste=[]
    for spiele in data_test:
        mannschaften= spiele[-2:]
        if team in mannschaften:
            home_team= spiele[:19]
            away_team= spiele[32:51]
            odds= spiele[64:67]
            output= spiele[67:70]
            game_data= np.concatenate((home_team,away_team,odds))
            liste.append(game_data)    
            return liste, output
        
"""        
input: outputs
convert output into single values, [1,0,0]=1,[0,1,0]=0,[0,0,1]=2
output: array with single values instead of single lists
"""     
def out_data(outputs):
    output=[]
    for i in outputs:
        if i[0]==1:
            output.append(0)
        elif i[1]==1:
            output.append(1)
        else:
            output.append(2)
    return output    

"""
input: data, test data, list of team, team
get all data, so TimeSequence, Aux_data, Output for the 2.GAME DAY
is a special case because you can't build a real TimeSequence because you know only one game #from past
knowing from the past
output: All data: (TimeSequence, Aux, Output)*2 (for both teams), Key from team
"""
def getspieltag2(data,test,mannschaftsliste,mannschaft):
    
    #Mannschaften holen(Indices)
    current_team=similar(mannschaft,mannschaftsliste)
    liste_time_series=[]
    liste_aux=[]
    liste_time_series_a=[]
    liste_aux_a=[]
    output=[]
    output_a=[]
    for spiele in test:
        mannschaften=spiele[-2:]
        if mannschaftsliste.index(current_team) in mannschaften:
            current_teams= mannschaften
            break
    
    #Daten für Mannschaften holen
    for key,values in data.items():
        for i in current_teams:
            if key==i and i == current_teams[0]:
                liste_time_series.append(values[0][19:32])
                liste_time_series.append(values[0][51:64])
                liste_aux.append(values[0][:19])
                liste_aux.append(values[0][32:51])
                liste_aux.append(values[0][64:67])
                output.append(values[0][67:70])
            if key==i and i == current_teams[1]:
                liste_time_series_a.append(values[0][19:32])
                liste_time_series_a.append(values[0][51:64])
                liste_aux_a.append(values[0][:19])
                liste_aux_a.append(values[0][32:51])
                liste_aux_a.append(values[0][64:67])
                output_a.append(values[0][67:70])
    liste_time_series= list(chain.from_iterable(liste_time_series))
    liste_aux= list(chain.from_iterable(liste_aux))
    liste_time_series_a= list(chain.from_iterable(liste_time_series_a))
    liste_aux_a= list(chain.from_iterable(liste_aux_a))
    
     
    return np.array([[liste_time_series]]), np.array([liste_aux]),np.array(output),np.array([[liste_time_series_a]]),np.array([liste_aux_a]), np.array(output_a), mannschaftsliste.index(current_team)

#---------------------SPIELTAG1
"""
input: team name, list of team, test data
match day 1
Here you have no TIMESEQUENCES --> create placeholder (-1)
get auxdata completelynormal and at predict again the same input as at fit
output: Team1,Team2, Prediction

"""
def spieltag1(teamname,mannschaftsliste,test):
    current_team=similar(teamname,mannschaftsliste)
    key= mannschaftsliste.index(current_team)
    aux_data_test,output_test=auxilary_data_test(test,key)
    aux_data= np.array(aux_data_test)
    time= np.array([[[-1.0]*26]*5]*1) #np.full

    out= output1(test,key)

    return aux_data, time, out

"""
input: test, team
Output conversion for the first match day
output: convert output
"""
def output1(test, mannschaft):
    for i in test:
        if mannschaft in i[-2:]:
            out= i[-5:-2]
            break
    if out[0]==1:
        return np.array([0])
    elif out[1]==1:
        return np.array([1])
    else: 
        return np.array([2])

In [4]:

"""
Building the model
We have several layers here: 
Lstm layer and then a Dense layer, Dense layer for aux, Dense for the merge.
denseActivation="relu"
output: model
"""
def build_model(aux_data, lstm_values, dropout=True, denseActivation='relu', units=32, lr=5e-4):
    
    inputDimension1 = np.shape(aux_data)[1]
    inputDimension2_0 = lstm_values[0].shape[0]
    inputDimension2_1 = lstm_values[0].shape[1]
    
    #Erstellung der auxiliary Schicht
    input1 = tf.keras.Input(shape=(inputDimension1,))#init eines tensors
    x1 = tf.keras.layers.Dense(units=units, activation=denseActivation)(input1)

    #Erstellung der LSTM Schicht
    input2 = tf.keras.Input(shape=(inputDimension2_0,inputDimension2_1))#init eines tensors
    x2 = tf.keras.layers.LSTM(units)(input2)
    
    #Erstellung der Schicht der LSTM Schicht für concat
    if dropout:
        x2 = tf.keras.layers.Dropout(0.1)(x2)
    x2 = tf.keras.layers.Dense(units=units, activation=denseActivation)(x2)
    
    #Erstellung der konkatenieren Schicht
    x = tf.keras.layers.concatenate([x1, x2], axis=1)
    
    #Erstellung der Output-Schicht
    for j in range(5,2,-1):
        for _ in range(3):
            x = tf.keras.layers.Dense(units=2**j, activation=denseActivation)(x)    
    outUnits=3
    out = tf.keras.layers.Dense(units=outUnits, activation='softmax')(x)
    
    #Erstellung des Modells
    inpts = [input1, input2]
    model = tf.keras.Model(inputs=inpts, outputs=[out])
    
    modelLoss = tf.keras.losses.SparseCategoricalCrossentropy()
    model.compile(loss=modelLoss,optimizer=tf.keras.optimizers.Adam(lr=lr),metrics=['accuracy'])
    return model


In [5]:
"""
input: matchday, team, typ
create data for the model
output: data for model 
"""

def model_data(spieltag,mannschaft,typ):
    
    #Spieltag und Mannschaft, für den die Daten gebildet werden sollen
    spieltag=spieltag
    teamname=mannschaft
    
    #Trainings- oder Validationssatz verwenden
    if typ == "t":
        training, test, mannschaftsliste= getdataTraining(spieltag)
    else:
        training, test, mannschaftsliste= getdata(spieltag)
    
    #Gegner und Heim und Gastmannschaft bestimmen
    teamname_gegner,ort= getgegner(test,teamname,mannschaftsliste)
    
    #Daten für Spieltag 1 oder 2 holen
    #Sonderfälle, da die Zeitreihen nicht normal erstellt werden können
    if spieltag==1:
            aux,time,out= spieltag1(teamname,mannschaftsliste,test)
            return aux, time,out
    elif spieltag==2:
        lstm_values, aux_data, outputs,lstm_values_a,aux_data_a,outputs_a,key= getspieltag2(training,test,mannschaftsliste,teamname)
        output_data=out_data(outputs)
        output_data_a=out_data(outputs_a)

    #----Normalfall----
    else:
    
        absteiger={0:[9,14],1:[7,8]}
        aufsteiger={1:[18,19],2:[9,14]}
        training_neu,spieltag_neu= data_auf_absteiger(training,mannschaftsliste,[teamname,teamname_gegner],spieltag,absteiger,aufsteiger)
        if training_neu=="KEY ERROR":
            return "Fehler: Mannschaft befindet sich nicht in der Saison"
        
        if spieltag_neu==1:
            aux,time,out= spieltag1(teamname,mannschaftsliste,test)
            return aux, time,out
            if ort=="home":
                return [teamname,teamname_gegner,predict,output_test] 
            else: 
                return [teamname_gegner,teamname,predict,output_test] 
        elif spieltag_neu==2:
            lstm_values, aux_data, outputs,lstm_values_a,aux_data_a,outputs_a,key= getspieltag2(training,test,mannschaftsliste,teamname)
            output_data=out_data(outputs)
            output_data_a=out_data(outputs_a)

        else:
            #Team
            lstm_values,key= time_sequence(training_neu,mannschaftsliste,teamname,spieltag_neu)
            aux_data,outputs=auxilary_data(training_neu,key,spieltag_neu)
            output_data= out_data(outputs[0]) #Output umschreiben in 2([1,0,0]),1([0,1,0]),0([0,0,1])

            #Gegner
            lstm_values_a,key_a= time_sequence(training_neu,mannschaftsliste,teamname_gegner,spieltag_neu)
            aux_data_a,outputs_a=auxilary_data(training_neu,key_a,spieltag_neu)
            output_data_a= out_data(outputs_a[0]) #Output umschreiben in 2([1,0,0]),1([0,1,0]),0([0,0,1])
    
    #Daten in Array umformen
    aux_data=np.array(aux_data)[0]
    lstm_values=np.array(lstm_values)
    output_data=np.array(output_data)
    aux_data_a=np.array(aux_data_a)[0]
    lstm_values_a=np.array(lstm_values_a)
    output_data_a=np.array(output_data_a)

    #Zeitreihen füllen, wenn diese zu klein sind
    if lstm_values[0].shape[0]<5:
        
        size= 5-lstm_values[0].shape[0]
        anzahl=lstm_values.shape[0]
        lstm_values_neu= np.array(anzahl*[5*[[-1.0]*26]])
        lstm_values_a_neu= np.array(anzahl*[5*[[-1.0]*26]])
        
        for i in range(0,anzahl):
            werte= lstm_values[i]
            werte_a= lstm_values_a[i]
            lstm_values_neu[i]= np.concatenate((np.array(size*[[-1]*26]),werte),axis=0)
            lstm_values_a_neu[i]= np.concatenate((np.array(size*[[-1]*26]),werte_a),axis=0)
            
    else:
        lstm_values_neu= lstm_values
        lstm_values_a_neu= lstm_values_a
    
    if ort=="home" and spieltag >2:
        time_all=np.concatenate((lstm_values_neu,lstm_values_a_neu),axis=0)
        aux_all= np.concatenate((aux_data,aux_data_a),axis=0)
        out_all= np.concatenate((output_data,output_data_a),axis=0)
    elif ort=="away" and spieltag >2:
        time_all=np.concatenate((lstm_values_a_neu,lstm_values_neu),axis=0)
        aux_all= np.concatenate((aux_data_a,aux_data),axis=0)
        out_all= np.concatenate((output_data_a,output_data),axis=0)
    elif ort=="home" and spieltag==2:
        time_all=np.concatenate((lstm_values_neu,lstm_values_a_neu),axis=0)
        aux_all= np.concatenate((aux_data,aux_data_a),axis=0)
        out_all= np.concatenate((output_data,output_data_a),axis=0)
    elif ort=="away" and spieltag==2:
        time_all=np.concatenate((lstm_values_a_neu,lstm_values_neu),axis=0)
        aux_all= np.concatenate(([aux_data],[aux_data_a]),axis=0)
        out_all= np.concatenate((output_data_a,output_data),axis=0)   

    #Wenn Spiele beim Spieltag 2 nicht getrennt
    if aux_all.shape == (82,):
        aux_all= aux_all.reshape(2,41)  
        
    return aux_all, time_all, out_all

"""
collect data from seasons and concat
output: concat data
"""
def collect_data():
    teams= get_mannschaftenTraining()
    saison1= teams[:18]
    saison2= get_mannschaftenTraining()
    saison3= get_mannschaftenTraining()
    del saison2[9]
    del saison2[13]
    del saison3[7]
    del saison3[7]
    liste_aux=[]
    liste_time=[]
    liste_out=[]
    
    #1. Saison 
    for i in range(1,35):
        for j in saison1:
            a,t,o=model_data(i,j,"t")
            liste_aux.append(a)
            liste_time.append(t)
            liste_out.append(o)
    #2. Saison 
    for i in range(35,69):
        for j in saison2:
            a,t,o=model_data(i,j,"t")
            liste_aux.append(a)
            liste_time.append(t)
            liste_out.append(o)
    #3.Saison
    for i in range(69,103):
        for j in saison3:
            a,t,o=model_data(i,j,"t")
            liste_aux.append(a)
            liste_time.append(t)
            liste_out.append(o)
    
    #Concat der einzelnen Datentypen(Aux,Zeit,Ausgabe)
    a= liste_aux[0]
    for i in liste_aux[1:]:
        a= np.concatenate((a,i),axis=0)
    

    t= liste_time[0]
    for i in liste_time[1:]:
        t= np.concatenate((t,i),axis=0)
    

    o= liste_out[0]
    for i in liste_out[1:]:
        o= np.concatenate((o,i),axis=0)
    
    return a,t,o



In [7]:
"""
fit the model and save it
ouput: trained model
"""
def fit_with_vali_as_train():
    #Training
    aux_t,time_t,out_t= collect_data()
    aux_v,time_v,out_v= collect_data_V("Daten2019.pkl")

    aux_all = np.concatenate((aux_t,aux_v),axis=0)
    time_all= np.concatenate((time_t,time_v),axis=0)
    out_all= np.concatenate((out_t,out_v),axis=0)
    print(aux_all.shape, time_all.shape)
    
    model = build_model(aux_t,time_t,dropout=True, lr=2e-5)
    model.fit([aux_all,time_all], out_all,epochs=26)
    model.save('LSTM4Saisons')
#fit_with_vali_as_train()

In [14]:
"""
input: matchday, team
Get data for prediction and run it
Load model and predict on it
ouput: prediction
"""
def getPredictData(spieltag, mannschaft):
    spieltag=spieltag
    teamname=mannschaft
    training, test, mannschaftsliste= getdata(spieltag)
    
    #Sonedrfall
    if spieltag==1:
        aux_data_test,lstm_values_test_neu,output_test= spieltag1(teamname,mannschaftsliste,test)
        new_model = tf.keras.models.load_model('LSTM4Saisons')
        output= new_model.predict([aux_data_test,lstm_values_test_neu])[0]

        teamname_gegner,ort= getgegner(test,teamname,mannschaftsliste)
        if output_test==[0]:
            output_test=[1,0,0]
        elif output_test==[1]:
            output_test=[0,1,0]
        else:
            output_test=[0,0,1]
        if ort=="home":
            return [teamname,teamname_gegner,output,output_test]
        else:
            return [teamname_gegner,teamname,output,output_test]
        
    #Hier wird nur der Key geholt    
    elif spieltag>2:
        lstm_values,key= time_sequence(training,mannschaftsliste,teamname,spieltag)
    else: 
        lstm_values, aux_data, outputs,lstm_values_a,aux_data_a,outputs_a,key= getspieltag2(training,test,mannschaftsliste,teamname)

    #Daten holen
    lstm_values_test= time_sequence_test(training,test,key,spieltag)
    aux_data_test, output_test=auxilary_data_test(test,key)
    lstm_values_test=np.array(lstm_values_test)
    aux_data_test=np.array(aux_data_test)
    #print(aux_data_test.shape,lstm_values_test.shape)
    #Zeitreihen füllen
    if lstm_values[0].shape[0]<5:
        size_test= 5-lstm_values_test[0].shape[0]
        anzahl_test= lstm_values_test[0].shape[0]
        lstm_values_test_neu= np.array(1*[5*[[-1.0]*26]])
        lstm_values_test_neu[0]= np.concatenate((np.array(size_test*[[-1]*26]),lstm_values_test[0]),axis=0)
    
    else:
        lstm_values_test_neu= lstm_values_test
    
    #Modell ladenund vorhersagen
    
    new_model = tf.keras.models.load_model('LSTM4Saisons')
    output= new_model.predict([aux_data_test,lstm_values_test_neu])[0]
    
    teamname_gegner,ort= getgegner(test,teamname,mannschaftsliste)
    
    #Ausgabe
    if ort=="home":
        return [teamname,teamname_gegner,output,output_test]
    else:
        return [teamname_gegner,teamname,output,output_test]
    
getPredictData(10, "Leverkusen")

['Leverkusen',
 'TSG Hoffenheim',
 array([0.03447261, 0.07287853, 0.8926489 ], dtype=float32),
 array([0., 0., 1.])]